# Extract Raw Data

In [1]:
import pandas as pd
import yahoo_fin.stock_info as si
import os

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
SP500_company_data = pd.read_html(url)[0]
SP500_company_data.to_csv("companies_data.csv")

print(SP500_company_data['Symbol'][:10])
print('Total de empresas: ',len(SP500_company_data))

0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
5    ADBE
6     AMD
7     AES
8     AFL
9       A
Name: Symbol, dtype: object
Total de empresas:  503


To evaluate the performance of our model, we are going to use data from 01/01/2010 until 31/10/2023 for the training and from 01/11/2023 until 31/12/2023 for the testing.

### Train Data

In [3]:
SP500_train_data = pd.DataFrame()

for company in SP500_company_data['Symbol']:
    try:
        company_data = si.get_data(company, start_date="01/01/2013", end_date="11/01/2023", index_as_date = True, interval="1d")
        SP500_train_data = pd.concat([SP500_train_data, company_data])
    except:
        ...

SP500_train_data.to_csv("stocks_train_data.csv")

In [4]:
SP500_train_data

,open,high,low,close,adjclose,volume,ticker
2013-01-02,78.754181,79.255852,78.561874,79.247490,54.016663,3835213,MMM
2013-01-03,78.879601,79.372910,78.704010,79.155518,53.953945,3234702,MMM
2013-01-04,79.255852,79.832779,79.046822,79.740807,54.352905,3235060,MMM
2013-01-07,79.448158,80.041809,79.230766,79.841141,54.421295,3283977,MMM
2013-01-08,79.573578,80.058525,79.515053,79.849495,54.426987,3175978,MMM
...,...,...,...,...,...,...,...
2023-10-25,165.559998,165.990005,162.350006,163.669998,161.677780,1273600,ZTS
2023-10-26,162.270004,162.289993,157.880005,158.070007,156.145966,2891100,ZTS
2023-10-27,157.929993,157.929993,155.160004,156.029999,154.130798,2407100,ZTS
2023-10-30,157.130005,158.410004,153.520004,157.000000,155.088959,2756200,ZTS


### Test Data

In [5]:
SP500_test_data = pd.DataFrame()

for company in SP500_company_data['Symbol']:
    try:
        company_data = si.get_data(company, start_date="10/25/2023", end_date="12/31/2023", index_as_date = True, interval="1d")
        SP500_test_data = pd.concat([SP500_test_data, company_data])
    except:
        ...

SP500_test_data.to_csv("stocks_test_data.csv")

In [6]:
SP500_test_data.tail()

,open,high,low,close,adjclose,volume,ticker
2023-12-22,195.320007,195.910004,192.740005,194.979996,193.067825,1548400,ZTS
2023-12-26,194.880005,196.339996,194.089996,195.500000,193.582733,814600,ZTS
2023-12-27,195.410004,197.009995,194.740005,196.899994,194.968994,766400,ZTS
2023-12-28,197.619995,198.600006,196.529999,197.160004,195.226456,880100,ZTS
2023-12-29,196.679993,198.009995,196.250000,197.369995,195.434387,1007200,ZTS


### Verificar se as empresas são iguais em ambos 

In [7]:
# verificar se contem o msm numero de empresas
train_ind = SP500_train_data["ticker"].unique()
test_ind = SP500_test_data["ticker"].unique()

diff = False
for ticker in train_ind:
    if ticker not in test_ind:
        diff = True
        break

if diff: print("Datasets contem diferentes empresas")
else: print("Datasets contem as mesmas empresas, sendo o numero total: %d" % len(test_ind))

Datasets contem as mesmas empresas, sendo o numero total: 497


## Vamos decidir quais empresas usar

Vamos escolher os 5 melhores setores e as respetivas 30 melhores empresas com base no <b> crescimento mensal </b>

In [8]:
# carregar os dados
SP500_company_data = pd.read_csv("companies_data.csv", index_col=0)
SP500_test_data = pd.read_csv("stocks_test_data.csv", index_col=0)
SP500_test_data.index = pd.to_datetime(SP500_test_data.index)
SP500_train_data = pd.read_csv("stocks_train_data.csv", index_col=0)
SP500_train_data.index = pd.to_datetime(SP500_train_data.index)

In [9]:
for sector in SP500_company_data["GICS Sector"].unique():
    n_empresas = SP500_company_data[SP500_company_data["GICS Sector"] == sector]["GICS Sector"].count()
    print(f"Total de empresas no setor '{sector}' -> {n_empresas}")

Total de empresas no setor 'Industrials' -> 78
Total de empresas no setor 'Health Care' -> 62
Total de empresas no setor 'Information Technology' -> 69
Total de empresas no setor 'Utilities' -> 31
Total de empresas no setor 'Financials' -> 72
Total de empresas no setor 'Materials' -> 28
Total de empresas no setor 'Consumer Discretionary' -> 50
Total de empresas no setor 'Real Estate' -> 31
Total de empresas no setor 'Communication Services' -> 22
Total de empresas no setor 'Consumer Staples' -> 38
Total de empresas no setor 'Energy' -> 22


In [10]:
def get_growth(df):
    # retorna o crescimento mensal da empresa 
    monthly_growth = 0
    monthly = df['close'].resample('ME').agg(['first', 'last'])
    
    for _, row in monthly.iterrows():
        monthly_growth += ((row['last'] - row['first']) / row['first']) * 100
        
    return monthly_growth/len(monthly)

In [11]:
SP500_train_data

,open,high,low,close,adjclose,volume,ticker
2013-01-02,78.754181,79.255852,78.561874,79.247490,54.016663,3835213,MMM
2013-01-03,78.879601,79.372910,78.704010,79.155518,53.953945,3234702,MMM
2013-01-04,79.255852,79.832779,79.046822,79.740807,54.352905,3235060,MMM
2013-01-07,79.448158,80.041809,79.230766,79.841141,54.421295,3283977,MMM
2013-01-08,79.573578,80.058525,79.515053,79.849495,54.426987,3175978,MMM
...,...,...,...,...,...,...,...
2023-10-25,165.559998,165.990005,162.350006,163.669998,161.677780,1273600,ZTS
2023-10-26,162.270004,162.289993,157.880005,158.070007,156.145966,2891100,ZTS
2023-10-27,157.929993,157.929993,155.160004,156.029999,154.130798,2407100,ZTS
2023-10-30,157.130005,158.410004,153.520004,157.000000,155.088959,2756200,ZTS


In [12]:
growth = {}
sectors_growth = {}
for sector in SP500_company_data["GICS Sector"].unique():
    mean_growth_sector = 0
    sector_data = SP500_company_data[SP500_company_data["GICS Sector"] == sector]
    data = {}
    
    for company in sector_data["Symbol"]: 
        try:
            stock_growth = get_growth(SP500_train_data[SP500_train_data["ticker"]==company])
            mean_growth_sector += stock_growth
            data[company] = stock_growth
        except:
            ...
    growth[sector] = dict(sorted(data.items(), key=lambda item: item[1], reverse=True)) # ordenar por ordem decrescente
    sectors_growth[sector] = mean_growth_sector/len(data)
# ordenar em forma decrescente
sectors_growth = dict(sorted(sectors_growth.items(), key=lambda item: item[1], reverse=True))

### Identificar os melhores setores e respetivas melhores empresas

Identificamos quais os 5 melhores setores e as respetivas 30 melhores empresas

In [13]:
if not os.path.exists("dataset_by_sector/"):
    os.mkdir("dataset_by_sector/")
    
metadata = pd.DataFrame(columns=['ticker', 'sector', 'model'])
# metadata = metadata.append({'tinker': 'Tinker3', 'sector': 'Sector3', 'model': 'Model3'}, ignore_index=True)
j = 1
for sector,m_growth in sectors_growth.items():
    if j > 5: break
    i = 1
    print(f"{j}) Setor: {sector} (avg monthly growth: {m_growth:.4f}%) - all companies in sector")
    print("    < 30 best companies >")
    for company,data in growth[sector].items():
        if i > 30: break
        print(f"    {company} - avg monthly growth: {data:.4f}%")
        metadata.loc[len(metadata)] = [company,sector.replace(" ","_"),"por_definir"]
        # quando criarmos os modelos vamos associar a cada stock
        i += 1
    j += 1
metadata.to_csv(f"dataset_by_sector/metadata.csv")

1) Setor: Information Technology (avg monthly growth: 1.6906%) - all companies in sector
    < 30 best companies >
    PLTR - avg monthly growth: 4.8093%
    ENPH - avg monthly growth: 4.5056%
    NVDA - avg monthly growth: 4.3222%
    AMD - avg monthly growth: 3.2526%
    CRWD - avg monthly growth: 3.1527%
    ANET - avg monthly growth: 2.9773%
    AVGO - avg monthly growth: 2.8534%
    NOW - avg monthly growth: 2.7821%
    MPWR - avg monthly growth: 2.6723%
    PANW - avg monthly growth: 2.6632%
    SMCI - avg monthly growth: 2.6045%
    EPAM - avg monthly growth: 2.5880%
    FTNT - avg monthly growth: 2.4486%
    CDNS - avg monthly growth: 2.4176%
    LRCX - avg monthly growth: 2.3566%
    FICO - avg monthly growth: 2.3395%
    SNPS - avg monthly growth: 2.2480%
    ADBE - avg monthly growth: 2.1802%
    MU - avg monthly growth: 2.1777%
    AMAT - avg monthly growth: 2.0917%
    ZBRA - avg monthly growth: 2.0421%
    MSFT - avg monthly growth: 2.0040%
    KLAC - avg monthly growth: 

### Guardar os dados

Vamos agora guardar os dados das 30 melhores empresas por setor

In [14]:
if not os.path.exists("dataset_by_sector/train/"):
    os.mkdir("dataset_by_sector/train/")
    
if not os.path.exists("dataset_by_sector/test/"):
    os.mkdir("dataset_by_sector/test/")    
    
limit_sector = 1

for sector,m_growth in sectors_growth.items():
    if limit_sector > 5: break
    limit_companies = 1
    
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    
    for company,data in growth[sector].items():
        if limit_companies > 30: break

        df_train = pd.concat([df_train,SP500_train_data[SP500_train_data["ticker"]==company]])
        df_test = pd.concat([df_test,SP500_test_data[SP500_test_data["ticker"]==company]])
        limit_companies += 1

    df_train.to_csv(f"dataset_by_sector/train/{sector.replace(' ','_')}.csv")
    df_test.to_csv(f"dataset_by_sector/test/{sector.replace(' ','_')}.csv")
    limit_sector += 1


### Remover os datasets iniciais 

Remover os datasets que já não são necessários

In [15]:
del_files = ["companies_data.csv","stocks_train_data.csv","stocks_test_data.csv"]

for csv_file in del_files:
    if os.path.exists(csv_file):
        os.remove(csv_file)
        print(f"Ficheiro '{csv_file}' removido com sucesso.")
    else:
        print(f"Ficheiro '{csv_file}' não encontrado.")

Ficheiro 'companies_data.csv' removido com sucesso.
Ficheiro 'stocks_train_data.csv' removido com sucesso.
Ficheiro 'stocks_test_data.csv' removido com sucesso.


# Parte do chico

### Decidir empresas com melhor crescimento

In [16]:
file_path = "stocks_train_data.csv"  
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:

    data['Percentile Growth'] = ((data['close'] - data['open']) / data['open']) * 100

    unique_companies = data.loc[data.groupby('ticker')['Percentile Growth'].idxmax()]

    sorted_data = unique_companies.sort_values(by='Percentile Growth', ascending=False)

    top_20_companies = sorted_data.head(50)

    print(top_20_companies[['ticker', 'open', 'close', 'Percentile Growth']])

    top_20_companies.to_csv("top_20_companies_1day.csv", index=False)
else:
    print("Data is missing")



FileNotFoundError: [Errno 2] No such file or directory: 'stocks_train_data.csv'

In [35]:
file_path = "stocks_train_data.csv"
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:

    data['Percentual Crescimento Diário'] = ((data['close'] - data['open']) / data['open']) * 100

    media_crescimento = data.groupby('ticker')['Percentual Crescimento Diário'].mean().reset_index()

    media_crescimento.rename(columns={'Percentual Crescimento Diário': 'Média Crescimento Diário'}, inplace=True)

    top_50_empresas = media_crescimento.sort_values(by='Média Crescimento Diário', ascending=False).head(100)

    print(top_50_empresas)

    top_50_empresas.to_csv("top_50_empresas_daily average.csv", index=False)
else:
    print("Data is missing")


    ticker  Média Crescimento Diário
465   VLTO                  0.586928
83     CEG                  0.238511
73    CARR                  0.165847
438    TPL                  0.157252
3     ABNB                  0.119603
..     ...                       ...
220    HII                  0.058211
156    ELV                  0.057973
396    ROL                  0.057695
447     TT                  0.057579
119   CTVA                  0.057425

[100 rows x 2 columns]


In [36]:
file_path = "stocks_train_data.csv"  
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:
    grouped = data.groupby('ticker')

    summary = grouped.agg(
        First_Open=('open', 'first'),
        Last_Close=('close', 'last')
    ).reset_index()

    summary['Percentual Crescimento'] = ((summary['Last_Close'] - summary['First_Open']) / summary['First_Open']) * 100

    top_100_empresas = summary.sort_values(by='Percentual Crescimento', ascending=False).head(20)

    print(top_100_empresas)

    top_100_empresas.to_csv("top_100_empresas.csv", index=False)
else:
    print("Erro: As colunas necessárias ('open', 'close', 'ticker') estão ausentes nos dados.")


    ticker  First_Open   Last_Close  Percentual Crescimento
445   TSLA    1.266667   248.479996            19516.836476
341   NVDA    0.462750    49.521999            10601.674913
331   NFLX    7.931429   486.880005             6038.616515
43    AVGO    1.830000   111.625000             5999.726633
147   DXCM    2.052500   124.089996             5945.797601
46    AXON    4.460000   258.329987             5692.152116
460    URI    9.920000   573.419983             5680.443332
185   FICO   21.379999  1164.010010             5344.387537
438    TPL   10.146667   524.150024             5065.736186
141    DPZ    8.530000   412.230011             4732.708369
80    CDNS    6.010000   272.369995             4431.946502
347   ODFL    4.548148   202.664993             4355.989259
61    BLDR    3.930000   166.940002             4147.837140
390   REGN   24.240000   878.289978             3523.308524
285   LULU   15.435000   511.290009             3212.536408
193   FTNT    1.788000    58.529999     